## Multidisciplinary Design Optimization for LV4
Placeholder text

In [ ]:
#a start towards a simple forward Euler integration for a rocket trajectory
#F=ma
#F=thrust - drag - gravity
#m_wet(t)=m-mdot*dt
#a=dv/dt
#v=dx/dt
#D = 0.5 * rho * v^2 *C_d * A
#T = mdot*isp*g_0
import math as m

#physical constants
g_0 = 9.81
dt = 0.1

#LV design variables
C_d = .5
radius = ()/2*
A =  m.pi*r**2
m_dry = 
m_wet =
r = m_wet/(m_wet+m_dry)

#initial conditions
x=[0,0]
v=[0,0]
t=[0,0]

while v > 0:
    dx = x[-1] - x[-2]
    x.append(x[-1] + dx)
    dv = v[-1] - v[-2]
    v.append(v[-1] + dv)
    m = m_dry + m_wet
    thrust - drag - g_0
    m_wet -= mdot
        if m_wet > 0:
            thrust = mdot*isp*g_0
        else:
            thrust = 0
            mdot = 0
            
    t.append(t[-1] + dt)